# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

A sport agent managing football players throughout Malaysia, presently, has a soccer player moving from Johor state in Malaysia to another state which is Selangor. He plan to move to Shah Alam which one of the largest city in Selangor, which is around 500KM from Johor. His top requirement is that he want to live to a location where closer to the football/soccer field and may be few restaurants nearby.

A sport agent managing football players throught Malaysia, presently, has a soccer player moving from Johor, Malaysia to Selangor, Malaysia. He plan to move to Shah Alam which one of the largest city in Selnagor, which is around 500KM from Johor. His top requirement is that he want to live to a location where closer to the football/soccer field and may be few restaurants nearby.  

## Data <a name="data"></a>

Shah Alam is a city and the state capital of Selangor, Malaysia and situated within the Petaling District and a small portion of the neighboring Klang District. Shah Alam replaced Kuala Lumpur as the capital city of the state of Selangor in 1978 due to Kuala Lumpur's incorporation into a Federal Territory in 1974 and become the capital of Malaysia.

There are 56 sections or neighborhoods in total. Generally Shah Alam can be divided into three parts; the north, central and south parts. North Shah Alam consists of 18 Sections. The Central Shah Alam is where all the state administrative buildings and agencies are situated. It consists of Section 1 until Section 24. South Shah Alam consists of 12 sections.

References: https://en.wikipedia.org/wiki/Shah_Alam

Since there no data readily available digitally for Shah Alam city, to construct dataframe that consist of the city, neighborhood, latitude and longitude will done based on the Shah Alam section names describes in the Wikipedia above and the coordinate will be gathered using geolocator.geocode iterative using Python.

## Methodology <a name="methodology"></a>

Once the data is available in dataframe that include the city/Shah Alam, neighborhood/Sections, latitude and longitude as described above, Foursquare API is used to explore the venues of all the neighborhood/Sections within 1KM radius of Shah Alam city. The venues are group by categories and then K-Mean cluster algorithm is used to cluster the venues. Then we can examine each cluster to determines which cluster has the high concentration of football/soccer field. That would be the ideal place for the soccer player to stay in Shah Alam city.

<h3>The Battle of Shah Alam Neighborhoods </h3>

The code for finding the best place for a soccer player to stay at Shah Alam, Malaysia

In [2]:
#import numpy and panda
import numpy as np 
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                       

<h3> Data Gathering </h3>
    

Craete the dataframe that consist of Shah Alam City/Neighborhood/Lat/Long

In [131]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# show the dataframe
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Get coordination for Shah Alam Neighborhood/Section

In [132]:
# city of interest
city = 'Shah Alam, MY'
# number of neighborhoods - section X, where X is a number. There 36 sections
section = 36

# get the location of each neighborhoods
for x in range(1, section):
    address = 'Section ' + str(x) + ', ' + city
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address) 
    if (location is not None):
       neighborhood_sec = 'Section ' + str(x)
       neighborhood_lat = location.latitude
       neighborhood_lon = location.longitude
       neighborhoods = neighborhoods.append({'Borough': city,
                                          'Neighborhood': neighborhood_sec,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    else:
       print('No coordinate of {}.'.format(address))  # in case geolocator cannot find the location

No coordinate of Section 7, Shah Alam, MY.
No coordinate of Section 26, Shah Alam, MY.
No coordinate of Section 27, Shah Alam, MY.
No coordinate of Section 28, Shah Alam, MY.
No coordinate of Section 29, Shah Alam, MY.
No coordinate of Section 30, Shah Alam, MY.
No coordinate of Section 31, Shah Alam, MY.
No coordinate of Section 32, Shah Alam, MY.
No coordinate of Section 33, Shah Alam, MY.
No coordinate of Section 34, Shah Alam, MY.


In [133]:
# city of interest
city = 'Shah Alam, MY'
# number of neighborhoods/section UX, where Xis a number. There 20 sections.
section = 20  

# get the location of each neighborhoods
for x in range(1, section):
    address = 'Section U' + str(x) + ', ' + city
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address) 
    if (location is not None):
       neighborhood_sec = 'Section U' + str(x)
       neighborhood_lat = location.latitude
       neighborhood_lon = location.longitude
       neighborhoods = neighborhoods.append({'Borough': city,
                                          'Neighborhood': neighborhood_sec,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    else:
       print('No coordinate of {}.'.format(address))  # in case geolocator cannot find the location
    


<h3>Render Shah Alam city <h3>

To see the Shah Alam sections/neigbrhoods are correctly position in the map

In [190]:
#Shah Alam, Malaysia Lat & Long
city_address = 'Shah Alam, MY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city_address)    
latitude = (location.latitude - 0.1) # but the given coordinate is a bit off
longitude = location.longitude

print('The geograpical coordinate of Shah Alam, are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Shah Alam, are 2.9898294, 101.53116264843747.


In [191]:
# create map of Shah Alam using latitude and longitude values
map_ShahAlam = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ShahAlam)  
    
map_ShahAlam

let's simplify the above map and segment and cluster only the neighborhoods in Shah Alam city, Malaysia. So let's slice the original dataframe and create a new dataframe of the Shah Alam data

In [136]:
#shahalam_data 
ShahAlam_data = neighborhoods
ShahAlam_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,"Shah Alam, MY",Section 1,3.069430,101.500523
1,"Shah Alam, MY",Section 2,3.067846,101.511315
2,"Shah Alam, MY",Section 3,3.075397,101.507429
3,"Shah Alam, MY",Section 4,3.078586,101.511890
4,"Shah Alam, MY",Section 5,3.083228,101.516758


In [137]:
ShahAlam_data.shape

(44, 4)

Define Foursquare Credentials and Version

In [138]:
CLIENT_ID = 'QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK' # your Foursquare ID
CLIENT_SECRET = 'U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK
CLIENT_SECRET:U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2


In [16]:
ShahAlam_data.loc[3, 'Neighborhood']

'Section 4'

Get the neighborhood's latitude and longitude values.

In [17]:
neighborhood_latitude = ShahAlam_data.loc[3, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ShahAlam_data.loc[3, 'Longitude'] # neighborhood longitude value

neighborhood_name = ShahAlam_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Section 4 are 3.0785863, 101.5118898.


In [22]:
import requests # library to handle requests

Create the GET request URL. Name your URL url.

In [139]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QJAF2DMIN03XY20HHQMUI0DZ51S2ROBZTBC42ZN2IXD2LLGK&client_secret=U45IJQUY5YAKODVEQ2SOCVRQGMXLWOEU20GCCAXD3EK2QTQ2&v=20180605&ll=3.0785863,101.5118898&radius=1000&limit=100'

Send the GET request and examine the resutls

In [140]:
results = requests.get(url).json()
#results

In [141]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [142]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Taman Bukit S.U.K,Park,3.080670,101.513317
1,Spiced Pumpkin,Eastern European Restaurant,3.074995,101.509666
2,McCafé,Café,3.074360,101.510338
3,Taman Tasik Shah Alam,Park,3.073930,101.515707
4,Serai Thai Restaurant,Thai Restaurant,3.075783,101.509335


In [143]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

96 venues were returned by Foursquare.


In [144]:
#function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create a new dataframe called ShahAlam_venues for each neighborhood

In [185]:
ShahAlam_venues = getNearbyVenues(names=ShahAlam_data['Neighborhood'],
                                   latitudes=ShahAlam_data['Latitude'],
                                   longitudes=ShahAlam_data['Longitude']
                                  )

Section 1
Section 2
Section 3
Section 4
Section 5
Section 6
Section 8
Section 9
Section 10
Section 11
Section 12
Section 13
Section 14
Section 15
Section 16
Section 17
Section 18
Section 19
Section 20
Section 21
Section 22
Section 23
Section 24
Section 25
Section 35
Section U1
Section U2
Section U3
Section U4
Section U5
Section U6
Section U7
Section U8
Section U9
Section U10
Section U11
Section U12
Section U13
Section U14
Section U15
Section U16
Section U17
Section U18
Section U19


In [146]:
print(ShahAlam_venues.shape)
ShahAlam_venues.head()

(891, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Section 1,3.06943,101.500523,Padang Sintetik UiTM Shah Alam,3.065586,101.499007,Soccer Field
1,Section 1,3.06943,101.500523,Kolam Renang UiTM,3.069100,101.500448,Pool
2,Section 1,3.06943,101.500523,Pusat Sukan UiTM,3.067421,101.497327,College Stadium
3,Section 1,3.06943,101.500523,Bilik Dobi Kolej Perindu 1,3.066763,101.501002,Dry Cleaner
4,Section 1,3.06943,101.500523,Pasar Mawar,3.069301,101.496100,Grocery Store


Let's check how many venues were returned for each neighborhood

In [147]:
ShahAlam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Section 1,6,6,6,6,6,6
Section 10,65,65,65,65,65,65
Section 11,20,20,20,20,20,20
Section 12,23,23,23,23,23,23
Section 13,34,34,34,34,34,34
Section 14,19,19,19,19,19,19
Section 15,22,22,22,22,22,22
Section 16,16,16,16,16,16,16
Section 17,29,29,29,29,29,29


Let's find out how many unique categories can be curated from all the returned venues

In [148]:
print('There are {} uniques categories.'.format(len(ShahAlam_venues['Venue Category'].unique())))

There are 147 uniques categories.


## Analysis <a name="analysis"></a>

<h3>Analyze each neighborhoods<h3>

In [149]:
# one hot encoding
ShahAlam_onehot = pd.get_dummies(ShahAlam_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ShahAlam_onehot['Neighborhood'] = ShahAlam_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ShahAlam_onehot.columns[-1]] + list(ShahAlam_onehot.columns[:-1])
ShahAlam_onehot = ShahAlam_onehot[fixed_columns]

ShahAlam_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Supermarket,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Volleyball Court,Water Park
0,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Section 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
ShahAlam_onehot.shape

(891, 148)

In [151]:
ShahAlam_grouped = ShahAlam_onehot.groupby('Neighborhood').mean().reset_index()
ShahAlam_grouped

,Neighborhood,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Supermarket,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Volleyball Court,Water Park
0,Section 1,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
1,Section 10,0.000000,0.000000,0.000000,0.015385,0.00,0.076923,0.000000,0.015385,0.000000,...,0.000000,0.00,0.000000,0.00,0.015385,0.000000,0.00,0.0000,0.015385,0.000000
2,Section 11,0.000000,0.000000,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.05,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
3,Section 12,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.043478,0.00,0.043478,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
4,Section 13,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.029412,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.029412,0.00,0.0000,0.000000,0.000000
5,Section 14,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
6,Section 15,0.000000,0.000000,0.000000,0.000000,0.00,0.136364,0.000000,0.000000,0.045455,...,0.000000,0.00,0.045455,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
7,Section 16,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0625,0.000000,0.000000
8,Section 17,0.034483,0.000000,0.068966,0.000000,0.00,0.034483,0.000000,0.000000,0.000000,...,0.000000,0.00,0.068966,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000
9,Section 18,0.000000,0.000000,0.000000,0.047619,0.00,0.047619,0.000000,0.000000,0.000000,...,0.000000,0.00,0.095238,0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.000000


In [152]:
ShahAlam_grouped.shape

(44, 148)

Print each neighborhood along with the top 5 most common venues

In [153]:
num_top_venues = 5

for hood in ShahAlam_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ShahAlam_grouped[ShahAlam_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Section 1----
             venue  freq
0     Concert Hall  0.17
1  College Stadium  0.17
2             Pool  0.17
3      Dry Cleaner  0.17
4    Grocery Store  0.17


----Section 10----
                 venue  freq
0     Malay Restaurant  0.12
1                 Café  0.09
2     Asian Restaurant  0.08
3    Indian Restaurant  0.05
4  Japanese Restaurant  0.05


----Section 11----
              venue  freq
0  Malay Restaurant  0.15
1        Food Truck  0.10
2  Asian Restaurant  0.10
3        Food Court  0.10
4  Halal Restaurant  0.05


----Section 12----
               venue  freq
0               Pool  0.09
1         Food Court  0.09
2  Convenience Store  0.04
3           Pharmacy  0.04
4  Electronics Store  0.04


----Section 13----
                  venue  freq
0      Malay Restaurant  0.18
1           Pizza Place  0.09
2                  Café  0.09
3  Fast Food Restaurant  0.06
4     Convenience Store  0.06


----Section 14----
                venue  freq
0                Café  0.11

In [154]:
# sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ShahAlam_grouped['Neighborhood']

for ind in np.arange(ShahAlam_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ShahAlam_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Section 1,Soccer Field,Grocery Store,Concert Hall,Pool,Dry Cleaner,College Stadium,Golf Course,Gift Shop,Comfort Food Restaurant,Gym Pool
1,Section 10,Malay Restaurant,Café,Asian Restaurant,Japanese Restaurant,Indian Restaurant,Electronics Store,Middle Eastern Restaurant,Pet Store,Ice Cream Shop,Bakery
2,Section 11,Malay Restaurant,Food Truck,Asian Restaurant,Food Court,Halal Restaurant,Convenience Store,Café,Electronics Store,Restaurant,Road
3,Section 12,Food Court,Pool,Other Great Outdoors,Bubble Tea Shop,Scenic Lookout,Malay Restaurant,Fast Food Restaurant,Karaoke Bar,Basketball Court,Baseball Field
4,Section 13,Malay Restaurant,Pizza Place,Café,Convenience Store,Racetrack,Skate Park,Fast Food Restaurant,Gym,Diner,Electronics Store


<h3>Part 2 -Cluster Neighborhoods</h3>
   

Cluster the Shah Alam City Neighborhoods using K_mean algorithm and find the best cluster for the soccer player

In [171]:
# set number of clusters
kclusters = 6

ShahAlam_grouped_clustering = ShahAlam_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ShahAlam_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 2, 1, 2, 2, 2, 2], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [172]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ShahAlam_merged = ShahAlam_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ShahAlam_merged = ShahAlam_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [158]:
ShahAlam_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Shah Alam, MY",Section 1,3.069430,101.500523,1,Soccer Field,Grocery Store,Concert Hall,Pool,Dry Cleaner,College Stadium,Golf Course,Gift Shop,Comfort Food Restaurant,Gym Pool
1,"Shah Alam, MY",Section 2,3.067846,101.511315,1,Café,Water Park,Asian Restaurant,Print Shop,Street Art,Athletics & Sports,College Academic Building,Arts & Crafts Store,Flower Shop,Deli / Bodega
2,"Shah Alam, MY",Section 3,3.075397,101.507429,2,Fast Food Restaurant,Malay Restaurant,Burger Joint,Ice Cream Shop,Asian Restaurant,Indian Restaurant,Food Truck,Convenience Store,Café,Arts & Crafts Store
3,"Shah Alam, MY",Section 4,3.078586,101.511890,2,Fast Food Restaurant,Malay Restaurant,Gym / Fitness Center,Asian Restaurant,Park,Indian Restaurant,Bakery,Eastern European Restaurant,Business Service,Building
4,"Shah Alam, MY",Section 5,3.083228,101.516758,1,Soccer Stadium,Soccer Field,Malay Restaurant,Coffee Shop,Café,Park,Gift Shop,Farmers Market,Concert Hall,Gym / Fitness Center


In [173]:
#Shah Alam, Malaysia Lat & Long
city_address = 'Shah Alam, MY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city_address)    
latitude = (location.latitude - 0.08) # but the given coordinate is a bit off
longitude = location.longitude

print('The geograpical coordinate of Shah Alam, are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Shah Alam, are 3.0098294, 101.53116264843747.


In [74]:
import math

 let's visualize the resulting clusters

In [174]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]  

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ShahAlam_merged['Latitude'], ShahAlam_merged['Longitude'], ShahAlam_merged['Neighborhood'], ShahAlam_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if not(math.isnan(cluster)):
        folium.CircleMarker(
           [lat, lon],
           radius=5,
           popup=label,
           color=rainbow[int(cluster-1)],
           fill=True,
           fill_color=rainbow[int(cluster-1)],
           fill_opacity=0.7).add_to(map_clusters)      
map_clusters

## Results and Discussion <a name="results"></a>

Our analysis data based on the neighborhoos cluster in Shah Alam show that most of the socces fileds are located at Cluser 2. Although there is a great number of restaurants in cluster 1 and cluster 3 which at the center of Shah Alam city but there is only one soccer field in cluser 3 and none in other clusters.

Result of all this is 6 clusters below show that there are only 2 potential locations for the socces player to move which is cluster 2 and cluster 3, which both clusers have a soccer field and restautants as shown below by examining detail on the clusters data.

<b>Cluster 1</b>

In [181]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 0, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Section 20,Malay Restaurant,Asian Restaurant,Pizza Place,Restaurant,Thai Restaurant,Café,Convenience Store,Department Store,Deli / Bodega,Dessert Shop
26,Section U2,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
27,Section U3,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
28,Section U4,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
29,Section U5,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
30,Section U6,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
31,Section U7,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
32,Section U8,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
33,Section U9,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store
34,Section U10,Malay Restaurant,Café,Restaurant,Halal Restaurant,Donut Shop,Ethiopian Restaurant,Building,Seafood Restaurant,Mediterranean Restaurant,Convenience Store


<b>cluster 2</b>

In [182]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 1, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Section 1,Soccer Field,Grocery Store,Concert Hall,Pool,Dry Cleaner,College Stadium,Golf Course,Gift Shop,Comfort Food Restaurant,Gym Pool
1,Section 2,Café,Water Park,Asian Restaurant,Print Shop,Street Art,Athletics & Sports,College Academic Building,Arts & Crafts Store,Flower Shop,Deli / Bodega
4,Section 5,Soccer Stadium,Soccer Field,Malay Restaurant,Coffee Shop,Café,Park,Gift Shop,Farmers Market,Concert Hall,Gym / Fitness Center
12,Section 14,Malay Restaurant,Halal Restaurant,Café,Indian Restaurant,Seafood Restaurant,Brazilian Restaurant,Boutique,Middle Eastern Restaurant,Sporting Goods Shop,Steakhouse
17,Section 19,Soccer Field,Dessert Shop,Farmers Market,Smoke Shop,Furniture / Home Store,Platform,Skate Park,Convenience Store,Deli / Bodega,Department Store


<b>cluster 3</b>

In [183]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 2, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Section 3,Fast Food Restaurant,Malay Restaurant,Burger Joint,Ice Cream Shop,Asian Restaurant,Indian Restaurant,Food Truck,Convenience Store,Café,Arts & Crafts Store
3,Section 4,Fast Food Restaurant,Malay Restaurant,Gym / Fitness Center,Asian Restaurant,Park,Indian Restaurant,Bakery,Eastern European Restaurant,Business Service,Building
5,Section 6,Malay Restaurant,Asian Restaurant,Convenience Store,Martial Arts Dojo,Food Court,Farmers Market,Soccer Field,Soccer Stadium,Bistro,Electronics Store
6,Section 8,Burger Joint,Convenience Store,Pizza Place,Indian Restaurant,Park,Market,Malay Restaurant,Bakery,Clothing Store,Playground
7,Section 9,Café,Asian Restaurant,Malay Restaurant,Indian Restaurant,Japanese Restaurant,Bakery,Ice Cream Shop,Middle Eastern Restaurant,Farmers Market,Coffee Shop
8,Section 10,Malay Restaurant,Café,Asian Restaurant,Japanese Restaurant,Indian Restaurant,Electronics Store,Middle Eastern Restaurant,Pet Store,Ice Cream Shop,Bakery
9,Section 11,Malay Restaurant,Food Truck,Asian Restaurant,Food Court,Halal Restaurant,Convenience Store,Café,Electronics Store,Restaurant,Road
10,Section 12,Food Court,Pool,Other Great Outdoors,Bubble Tea Shop,Scenic Lookout,Malay Restaurant,Fast Food Restaurant,Karaoke Bar,Basketball Court,Baseball Field
11,Section 13,Malay Restaurant,Pizza Place,Café,Convenience Store,Racetrack,Skate Park,Fast Food Restaurant,Gym,Diner,Electronics Store
13,Section 15,Malay Restaurant,Asian Restaurant,Convenience Store,Office,Playground,Breakfast Spot,Boutique,Restaurant,Café,Food


<b>cluster 4</b>

In [178]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 3, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Section 21,Chinese Restaurant,Cemetery,Campground,Water Park,Dry Cleaner,Fast Food Restaurant,Farmers Market,Exhibit,Ethiopian Restaurant,Electronics Store


<b>cluster 5</b>

In [179]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 4, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Section 35,Art Gallery,Lake,BBQ Joint,Basketball Court,Exhibit,Water Park,Ethiopian Restaurant,Flea Market,Field,Fast Food Restaurant


<b>cluster 6</b>

In [180]:
ShahAlam_merged.loc[ShahAlam_merged['Cluster Labels'] == 5, ShahAlam_merged.columns[[1] + list(range(5, ShahAlam_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Section 22,Board Shop,Food Court,Malay Restaurant,Water Park,Flower Shop,Field,Fast Food Restaurant,Farmers Market,Exhibit,Ethiopian Restaurant


## Conclusion <a name="conclusion"></a>

Purpose of this project was to help a sport agent managing football players throughout Malaysia, to identify the best potential location for a soccer player moving from Johor state in Malaysia to another state which is Selangor. He plan to move to Shah Alam which one of the largest city in Selangor, which is around 500KM from Johor. His top requirement is that he want to live to a location where closer to the football/soccer field and may be few restaurants nearby.

Foursquare API is used to explore the venues of all the neighborhood/Sections within 1KM radius of Shah Alam city. The venues are group by categories and then K-Mean cluster algorithm is used to cluster the venues. Then we can examine each cluster to determines which cluster has the high concentration of football/soccer field. That would be the ideal place for the soccer player to stay in Shah Alam city.

Final decission on optimal location will be made by sport agent based on specific characteristics of neighborhoods and locations in every recommended location based on the clusters that met the criteria which are location where closer to the football/soccer field and may be few restaurants nearby. Based on the result and discussion there are only 2 pontential location which are cluster 2 and cluster 3.